# **Code to derive the debris cover metrics from "The state of rock debris covering Earth's glaciers" by Herreid and Pellicciotti, Nature Geoscience, 2020**  
Version 1.0 (this is currently a work in progress [26 Aug 2021] and missing some metrics!)

By Sam Herreid (samherreid@gmail.com)
      
To run the entire code, required input data are:

* Glacier outline(s) (.shp)
* Debris cover outlines (.shp) **OR** Landsat TM, ETM+, OLI image(s) (.tif, do not change default NASA file names)

# Define input data directories

In [ ]:
glacierOutlines_dir = r"SamRGI_outlines"

In [ ]:
debrisOutlines_dir = r"SamRGI_debris"

In [ ]:
Landsat_dir = r""

Of the above datasets, pick which you would like to have be the template coordinate system. Or define a new one. Or, if you are sure they are in the same coordinate system, you can skip this O(n) space/time operation.

Accepted arguments are: 

* *glacier_shp*
* *debris_shp*
* *skip*
* define a spatial reference with "arcpy.SpatialReference(32145)" where 32145 is replaced with a factory code for the desired spatial reference which can be found for [geographic coordinate systems](https://pro.arcgis.com/en/pro-app/arcpy/classes/pdf/geographic_coordinate_systems.pdf) or [projected coordinate systems](https://pro.arcgis.com/en/pro-app/arcpy/classes/pdf/projected_coordinate_systems.pdf). More information [here](https://pro.arcgis.com/en/pro-app/arcpy/classes/spatialreference.htm).

In [ ]:
spatialRef = "glacier_shp"

# Define parameters

**ablationZone**:

Exclude small debris patches from ablation zone derivation (number of 30m pixels) "20" used in Herreid and Pellicciotti (2020)

In [ ]:
n_deb_pixels_cut_ablationZone = 20

"0.75" used in Herreid and Pellicciotti, Nature Geoscience, 2020.

In [ ]:
x_val = 0.75

# Import packages

In [ ]:
import os
import arcpy
#local
import commonCoords

# Set up data

Get lists of glacierized regions and associated debris shapefiles:

In [ ]:
#env.workspace = workspace

regionList_glacier = []
regionList_debris = []
for gl_file in os.listdir(glacierOutlines_dir):
    if gl_file.endswith(".shp"):
        regionList_glacier.append(glacierOutlines_dir + "\\" + gl_file)
        for deb_file in os.listdir(debrisOutlines_dir):
            if deb_file.endswith(".shp"):
                if deb_file.startswith(gl_file.split(".")[0]):
                    regionList_debris.append(debrisOutlines_dir + "\\" + deb_file)
if len(regionList_glacier) != len(regionList_debris):
    print("Rename debris file(s) to have the same ID name at the beginning of the file as its associated glacier .shp. Make sure there is a debris file for each glacier file.")

Copy all of the input data into the workspace defined above and get all input data into the same projection and coordinate system:

In [ ]:
if not os.path.exists("working"):
        os.makedirs("working")
for i, glaciers in enumerate(regionList_glacier):
    workingFolder = "working\\"+glaciers.split("\\")[-1].split(".")[0]+"_working"
    if not os.path.exists(workingFolder):
        os.makedirs(workingFolder)
    commonCoords.commonCoords(glaciers, regionList_debris[i], spatialRef, workingFolder)